# Задание для DS по разработке PD модели

## Необходиом разработать модель предсказания дефолта. Даны финансовые данные юридических лиц (выручка, активы, пассивы и т.д). 

### Импортируем библиотеки

In [18]:
import pandas as pd
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

### Подгружаем данные

In [2]:
!wget https://raw.githubusercontent.com/BKHV/ds-test/master/PD-data.csv

/bin/sh: wget: command not found


In [3]:
df = pd.read_csv('PD-data.csv', sep=';')

Размеры выборок

In [5]:
print(df.shape)

(32595, 37)


Количество разных типов фичей

In [6]:
df.dtypes.value_counts()

float64    25
int64      11
object      1
dtype: int64

Уровень дефолт-рейта в выборке = 6,7219%

In [9]:
df['default_12m'].values.mean()

0.06721889860408038

In [10]:
df.record_id.nunique()

32595

32595 уникальных наблюдений

### Строим бэйзлайн модель c 3-ми фичами

In [11]:
features = ['bus_age','ul_capital_sum', 'ul_founders_cnt']

In [12]:
features_df = df.loc[:,features]
features_df['default_12m'] = df['default_12m']

In [13]:
features_df.head()

,bus_age,ul_capital_sum,ul_founders_cnt,default_12m
0,198,50000.0,2,0
1,256,10000.0,5,0
2,33,10000.0,2,0
3,185,300000.0,1,0
4,220,10000.0,2,0


### Построим карту коллеряций

In [14]:
features_df.corr()

,bus_age,ul_capital_sum,ul_founders_cnt,default_12m
bus_age,1.000000,0.067037,0.209244,-0.111336
ul_capital_sum,0.067037,1.000000,0.029621,-0.019556
ul_founders_cnt,0.209244,0.029621,1.000000,-0.024054
default_12m,-0.111336,-0.019556,-0.024054,1.000000


In [19]:
X_train, X_test, y_train, y_test = train_test_split(features_df.drop('default_12m',axis=1),
                                                    features_df['default_12m'],
                                                    random_state=42, shuffle=True)

logit = LogisticRegression(C=1,solver='lbfgs',max_iter=1000)
cv_baseline = cross_val_score(logit, X_train, y_train, cv=5, scoring='roc_auc').mean()
cv_baseline

0.6488699458962778

ROC/AUC бэйзлайн модели - 64,89%

Для загрузки своего рузультата на https://dsbattle.com/hackathons/juniords/ вам нужно создать csv файл с двумя колонками (id, predict), разделитель - точка с запятой (;)